# **ENSTROPHY TESTING**

## **Installs**

### Firedrake

In [15]:
try:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-development-real.sh" -O "/tmp/firedrake-install.sh"
    !bash "/tmp/firedrake-install.sh"
    from firedrake import *  # noqa: F401
except:
    from firedrake import *  # noqa: F401

--2026-02-11 18:06:49--  https://fem-on-colab.github.io/releases/firedrake-install-development-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4775 (4.7K) [application/x-sh]
Saving to: ‘/tmp/firedrake-install.sh’

/tmp/firedrake-inst 100%[===================>]   4.66K  --.-KB/s    in 0s      

2026-02-11 18:06:49 (19.7 MB/s) - ‘/tmp/firedrake-install.sh’ saved [4775/4775]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FIREDRAKE_INSTALLED=/usr/local/share/fem-on-colab/firedrake.installed
+ [[ ! -f /usr/local/share/fem-on-colab/firedrake.installed ]]
+ set +x
























#########################################################

### Gmsh

In [16]:
try:
    !wget "https://fem-on-colab.github.io/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh"
    !bash "/tmp/gmsh-install.sh"
    import gmsh  # noqa: F401
except:
    import gmsh  # noqa: F401

--2026-02-11 18:06:49--  https://fem-on-colab.github.io/releases/gmsh-install.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3497 (3.4K) [application/x-sh]
Saving to: ‘/tmp/gmsh-install.sh’

/tmp/gmsh-install.s 100%[===================>]   3.42K  --.-KB/s    in 0s      

2026-02-11 18:06:49 (37.0 MB/s) - ‘/tmp/gmsh-install.sh’ saved [3497/3497]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ GMSH_INSTALLED=/usr/local/share/fem-on-colab/gmsh.installed
+ [[ ! -f /usr/local/share/fem-on-colab/gmsh.installed ]]
+ set +x
























################################################################################
#     This installation

## **Code**

### Mesh

In [17]:
def rect_with_hole_mesh(r=0.5, left=1.0, right=5.0, updown=1.0, h=0.2, obstacle="circle"):
    '''
    r = hole radius
    left = space to left of hole centre
    right = space to right of hole centre
    updown = space above / below hole centre
    h = (target) mesh size
    '''
    gmsh.initialize()
    gmsh.model.add("rect_with_hole")

    # Derived rectangle bounds
    xmin = -left
    xmax = right
    ymin = -updown
    ymax = updown


    # ---------------
    #    Rectangle
    # ---------------
    p1 = gmsh.model.geo.addPoint(xmin, ymin, 0, h)
    p2 = gmsh.model.geo.addPoint(xmax, ymin, 0, h)
    p3 = gmsh.model.geo.addPoint(xmax, ymax, 0, h)
    p4 = gmsh.model.geo.addPoint(xmin, ymax, 0, h)

    l_bottom = gmsh.model.geo.addLine(p1, p2)
    l_right  = gmsh.model.geo.addLine(p2, p3)
    l_top    = gmsh.model.geo.addLine(p3, p4)
    l_left   = gmsh.model.geo.addLine(p4, p1)

    rect_loop = gmsh.model.geo.addCurveLoop(
        [l_bottom, l_right, l_top, l_left]
    )


    # ------------------------------
    #    Hole (centred at origin)
    # ------------------------------
    if obstacle=="circle":
        c  = gmsh.model.geo.addPoint(0, 0, 0, h)

        cp = gmsh.model.geo.addPoint( r, 0, 0, h)
        cn = gmsh.model.geo.addPoint(-r, 0, 0, h)
        ct = gmsh.model.geo.addPoint(0,  r, 0, h)
        cb = gmsh.model.geo.addPoint(0, -r, 0, h)

        a1 = gmsh.model.geo.addCircleArc(cp, c, ct)
        a2 = gmsh.model.geo.addCircleArc(ct, c, cn)
        a3 = gmsh.model.geo.addCircleArc(cn, c, cb)
        a4 = gmsh.model.geo.addCircleArc(cb, c, cp)
    elif obstacle=="semicircle":
        c  = gmsh.model.geo.addPoint(0, 0, 0, h)

        cp = gmsh.model.geo.addPoint(0,  0, 0, h)
        cn = gmsh.model.geo.addPoint(-r, 0, 0, h)
        ct = gmsh.model.geo.addPoint(0,  r, 0, h)
        cb = gmsh.model.geo.addPoint(0, -r, 0, h)

        a1 = gmsh.model.geo.addLine(cp, ct)
        a2 = gmsh.model.geo.addCircleArc(ct, c, cn)
        a3 = gmsh.model.geo.addCircleArc(cn, c, cb)
        a4 = gmsh.model.geo.addLine(cb, cp)
    elif obstacle=="diamond":
        cp = gmsh.model.geo.addPoint( r, 0, 0, h)
        cn = gmsh.model.geo.addPoint(-r, 0, 0, h)
        ct = gmsh.model.geo.addPoint(0,  r, 0, h)
        cb = gmsh.model.geo.addPoint(0, -r, 0, h)

        a1 = gmsh.model.geo.addLine(cp, ct)
        a2 = gmsh.model.geo.addLine(ct, cn)
        a3 = gmsh.model.geo.addLine(cn, cb)
        a4 = gmsh.model.geo.addLine(cb, cp)
    elif obstacle=="wedge":
        cp = gmsh.model.geo.addPoint( 0, 0, 0, h)
        cn = gmsh.model.geo.addPoint(-r, 0, 0, h)
        ct = gmsh.model.geo.addPoint(0,  r, 0, h)
        cb = gmsh.model.geo.addPoint(0, -r, 0, h)

        a1 = gmsh.model.geo.addLine(cp, ct)
        a2 = gmsh.model.geo.addLine(ct, cn)
        a3 = gmsh.model.geo.addLine(cn, cb)
        a4 = gmsh.model.geo.addLine(cb, cp)
    elif obstacle=="square":
        cp = gmsh.model.geo.addPoint( r,  r, 0, h)
        cn = gmsh.model.geo.addPoint(-r, -r, 0, h)
        ct = gmsh.model.geo.addPoint(-r,  r, 0, h)
        cb = gmsh.model.geo.addPoint(r,  -r, 0, h)

        a1 = gmsh.model.geo.addLine(cp, ct)
        a2 = gmsh.model.geo.addLine(ct, cn)
        a3 = gmsh.model.geo.addLine(cn, cb)
        a4 = gmsh.model.geo.addLine(cb, cp)

    if obstacle!="none":
        hole_loop = gmsh.model.geo.addCurveLoop([a1, a2, a3, a4])


    # -----------------------
    #    Surface with hole
    # -----------------------
    if obstacle!="none":
        surface = gmsh.model.geo.addPlaneSurface([rect_loop, hole_loop])
    else:
        surface = gmsh.model.geo.addPlaneSurface([rect_loop])

    gmsh.model.geo.synchronize()


    # ---------------------
    #    Physical groups
    # ---------------------
    gmsh.model.addPhysicalGroup(1, [l_left], tag=1)
    gmsh.model.setPhysicalName(1, 1, "inlet")

    gmsh.model.addPhysicalGroup(1, [l_right], tag=2)
    gmsh.model.setPhysicalName(1, 2, "outlet")

    gmsh.model.addPhysicalGroup(1, [l_top, l_bottom], tag=3)
    gmsh.model.setPhysicalName(1, 3, "wall")

    if obstacle!="none":
        gmsh.model.addPhysicalGroup(1, [a1, a2, a3, a4], tag=4)
        gmsh.model.setPhysicalName(1, 4, "hole")

    gmsh.model.addPhysicalGroup(2, [surface], tag=5)


    # ----------
    #    Mesh
    # ----------
    gmsh.model.mesh.generate(2)
    gmsh.write("rect_with_hole.msh")
    gmsh.finalize()

    return Mesh("rect_with_hole.msh")

### Navier–Stokes

In [24]:
def navier_stokes(h=0.2, Re=1.0, sigma=10.0, gamma=1.0, dt=0.02, T=0.1, save=True):
    # -----------
    #    Setup
    # -----------
    # Mesh
    mesh = rect_with_hole_mesh(h=h, obstacle="circle")
    n = FacetNormal(mesh)
    x, y = SpatialCoordinate(mesh)

    # Normalise parameters
    h_c = Constant(h)
    Re_c = Constant(Re)
    sigma_c = Constant(sigma)
    gamma_c = Constant(gamma)
    dt_c = Constant(dt)

    # Spaces
    W = FunctionSpace(mesh, "CG", 3)
    V = FunctionSpace(mesh, "BDM", 2)
    Q = FunctionSpace(mesh, "DG", 1)
    Z = W * V * Q

    # Unknowns
    z = Function(Z)
    w, u, p = split(z)
    w_sol, u_sol, p_sol = z.subfunctions
    w_sol.rename("vorticity")
    u_sol.rename("velocity")
    p_sol.rename("pressure")

    # Tracker functions
    u_old = Function(V)

    # Tests
    c, v, q = TestFunctions(Z)

    # Solver parameters
    sp = {
        "snes_monitor" : None,
        "snes_converged_reason" : None,
        # "snes_atol" : 1e-10,
        # "snes_rtol" : 1e-10,
    }


    # --------------------------
    #    Initial Stokes solve
    # --------------------------
    # Operators
    def dev(w):
        return sym(grad(w))
    def jump(w):
        return outer(w('+') - w('-'), n('+'))
    def jump_dev(w):
        return sym(jump(w))
    def rotate(w):
        return as_vector([-w[1], w[0]])

    # Functions
    u_mid = (u + u_old) / 2

    # Vorticity coupling term
    vorticity_bulk = 1 / Re_c * inner(grad(w), grad(c)) * dx

    # Dissipation term (auxiliary)
    diss_aux_bulk = 2 / Re_c * inner(dev(u), dev(rotate(grad(c)))) * dx
    diss_aux_int = 2 / Re_c * (
      - inner(avg(dev(u)), jump_dev(rotate(grad(c))))
      - inner(jump_dev(u), avg(dev(rotate(grad(c)))))
      + sigma_c / h_c * inner(jump_dev(u), jump_dev(rotate(grad(c))))
    ) * dS
    diss_aux_ext = (
        gamma_c * inner(u, rotate(grad(c))) * (ds(1) + ds(4))  # Inlet + Obstacle
      - inner(p, dot(rotate(grad(c)), n)) * ds(2)  # Outlet
    )

    # Pressure term
    pressure_bulk = inner(p, div(v)) * dx

    # Dissipation term (primary)
    diss_bulk = 2 / Re_c * inner(dev(u), dev(v)) * dx
    diss_int = 2 / Re_c * (
      - inner(avg(dev(u)), jump_dev(v))
      - inner(jump_dev(u), avg(dev(v)))
      + sigma_c / h_c * inner(jump_dev(u), jump_dev(v))
    ) * dS
    diss_ext = gamma_c * inner(u, v) * (ds(1) + ds(4))  # Inlet + Obstacle

    # Incompressibility term
    incomp_bulk = inner(div(u), q) * dx

    # Residual
    F = (
        vorticity_bulk + (diss_aux_bulk + diss_aux_int + diss_aux_ext)
      - pressure_bulk + (diss_bulk + diss_int + diss_ext)
      - incomp_bulk
    )

    # Boundary conditions
    def partial_slip_omega_bc(index, radius=None):
        if radius is None:
            return EquationBC(inner(w + gamma_c*Re_c * dot(u, rotate(n)), c) * ds(index) == 0, z, index, V=Z.sub(0))
        else:
            return EquationBC(inner(w + (gamma_c*Re_c + 2/radius) * dot(u, rotate(n)), c) * ds(index) == 0, z, index, V=Z.sub(0))
    bcs = [
        partial_slip_omega_bc(1),  # Inlet
        DirichletBC(Z.sub(0), 0, 3),  # Walls
        partial_slip_omega_bc(4, radius=0.5),  # Obstacle
        DirichletBC(Z.sub(1), as_vector([1.0, 0.0]), 1),  # Inlet
        DirichletBC(Z.sub(1), as_vector([0.0, 0.0]), 3),  # Walls
        DirichletBC(Z.sub(1), as_vector([0.0, 0.0]), 4),  # Obstacle
    ]

    # PVD setup
    if save:
        !rm -rf velocity*
        outfile = VTKFile("velocity.pvd")

    # Solve
    print(GREEN % f"Setting up ICs for t = 0:")
    solve(F == 0, z, bcs=bcs, solver_parameters=sp)
    # if save: outfile.write(u_sol, p_sol)
    if save: outfile.write(w_sol, u_sol, p_sol)
    u_old.assign(u_sol)


    # ------------------------------
    #    Full Navier-Stokes solve
    # ------------------------------
    # Advecterm term (auxiliary)
    adv_aux_ext = 1/2 * inner(dot(u, u), dot(rotate(grad(c)), n)) * ds(2)  # Outlet

    # Transient term
    transient_bulk = 1 / dt_c * inner((u - u_old), v) * dx

    # Advective term (primary)
    adv_bulk = inner(w * rotate(u_mid), v) * dx
    adv_ext = 1/2 * inner(dot(u, u), dot(v, n)) * ds(2)  # Outlet

    # Residual
    F += (
        adv_aux_ext
      + transient_bulk + (adv_bulk + adv_ext)
    )

    # Time loop
    t = 0.0
    step = 0
    while step < round(T/dt):
        t += dt
        step += 1
        print(GREEN % f"Solving for t = {t:.4e}:")
        solve(F == 0, z, bcs=bcs, solver_parameters=sp)
        if save: outfile.write(w_sol, u_sol, p_sol)
        u_old.assign(u_sol)

In [25]:
def download():
    import zipfile
    import glob
    import os
    from google.colab import files

    # Zip the .pvd and all .vtu files
    zip_name = "velocity_data.zip"
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        if os.path.exists("velocity.pvd"):
            zipf.write("velocity.pvd")

        # The .vtu files are stored in a folder called 'velocity'
        if os.path.isdir("velocity"):
            for root, _, files_list in os.walk("velocity"):
                for file in files_list:
                    if file.endswith(".vtu"):
                        file_path = os.path.join(root, file)
                        zipf.write(file_path, arcname=file_path)

    files.download(zip_name)
    print(BLUE % f"Download complete!")

In [ ]:
# navier_stokes(h=0.2, Re=200.0, sigma=0.5, gamma=1.0, dt=0.2, T=5.0, save=True)
navier_stokes(h=0.1, Re=500.0, sigma=0.5, gamma=1.0, dt=0.2, T=20.0, save=True)
download()

Setting up ICs for t = 0:
  0 SNES Function norm 4.605968585890e+01
  1 SNES Function norm 8.313501181487e-03
